Este script emula el funcionamiento diario de un parqueadero bajo sus principales procesos:

* **Operario**: almacena la información de quien supervisa el turno (cédula, nombre, correo y rol).
* **Celdas**: dispone de 10 espacios para carros y 6 para motos; cada una lleva registro de su tipo y estado (ocupada o libre).
* **Vehículos**: se registran con placa, tipo (carro o moto), marca y color.
* **Tiquete de ingreso**: al aparcar, se genera un comprobante que incluye los datos del vehículo, la celda asignada y la hora de entrada.
* **Tiquete de salida**: al retirar el vehículo, el comprobante muestra la hora de salida, el método de pago y el monto a pagar (tarifa por hora completa: 3 500 COP para carros y 2 000 COP para motos).
* **Estado del parqueadero**: después de cada ingreso o salida, se actualiza y muestra cuántas celdas de carro y de moto están ocupadas y cuántas están disponibles.
* **Flujo de la simulación**: en `main()` se presenta el estado inicial, se procesan múltiples ingresos (incluyendo una placa duplicada que genera alerta) y varias salidas con diferentes métodos de pago.


In [8]:
import time
from datetime import datetime, timedelta

class Operator:
    def __init__(self, id_number, full_name, email, role):
        self.id_number = id_number
        self.full_name = full_name
        self.email = email
        self.role = role

class Cell:
    def __init__(self, cell_id, cell_type):
        self.cell_id = cell_id
        self.cell_type = cell_type  # 'carro' o 'moto'
        self.occupied = False
        self.vehicle = None

class Vehicle:
    def __init__(self, license_plate, vehicle_type, brand, color, owner=None):
        self.license_plate = license_plate
        self.vehicle_type = vehicle_type  # 'carro' o 'moto'
        self.brand = brand
        self.color = color
        self.owner = owner

class Ticket:
    def __init__(self, vehicle, cell, entry_time):
        self.vehicle = vehicle
        self.cell = cell
        self.entry_time = entry_time
        self.exit_time = None
        self.paid = False
        self.payment_method = None

    def calculate_fee(self):
        if not self.exit_time:
            return 0
        rate = 3500 if self.vehicle.vehicle_type == 'carro' else 2000
        total_seconds = (self.exit_time - self.entry_time).total_seconds()
        hours = int(total_seconds // 3600)
        minutes = int((total_seconds % 3600) // 60)
        # cobramos al menos 1 hora, y cada minuto extra cuenta como hora completa
        hours += 1 if minutes > 0 else 0
        hours = max(1, hours)
        return hours * rate

    def pretty_print(self, exit=False):
        title = "TIQUETE DE SALIDA" if exit else "TIQUETE DE INGRESO"
        entry_str = self.entry_time.strftime('%H:%M')
        print(f"\n—— {title} ——")
        print(f"Tipo Vehículo : {self.vehicle.vehicle_type.capitalize()}")
        print(f"Placa         : {self.vehicle.license_plate}")
        print(f"Marca         : {self.vehicle.brand}")
        print(f"Color         : {self.vehicle.color}")
        print(f"Celda         : {self.cell.cell_id} ({self.cell.cell_type})")
        print(f"Entrada       : {entry_str}")
        if exit:
            exit_str = self.exit_time.strftime('%H:%M')
            fee = self.calculate_fee()
            print(f"Salida        : {exit_str}")
            print(f"Método Pago   : {self.payment_method}")
            print(f"Total a Pagar : {fee} COP")
        print("———————————————\n")

class ParkingLot:
    def __init__(self, operator, car_cells=10, moto_cells=6):
        self.operator = operator
        self.cells = []
        for i in range(car_cells):
            self.cells.append(Cell(i+1, 'carro'))
        for j in range(moto_cells):
            self.cells.append(Cell(car_cells + j+1, 'moto'))
        self.tickets = {}

    def available_cells(self, vehicle_type=None):
        cells = [c for c in self.cells if not c.occupied]
        if vehicle_type:
            cells = [c for c in cells if c.cell_type == vehicle_type]
        return cells

    def occupied_cells(self):
        return [c for c in self.cells if c.occupied]

    def report_status(self):
        cars_free = self.available_cells('carro')
        motos_free = self.available_cells('moto')
        cars_occ = [c for c in self.occupied_cells() if c.cell_type == 'carro']
        motos_occ = [c for c in self.occupied_cells() if c.cell_type == 'moto']
        print("Estado actual del parqueadero:")
        print(f"  Celdas disponibles para CARRO ({len(cars_free)}) : {[c.cell_id for c in cars_free]}")
        print(f"  Celdas ocupadas para CARRO  ({len(cars_occ)}) : {[c.cell_id for c in cars_occ]}")
        print(f"  Celdas disponibles para MOTO  ({len(motos_free)}) : {[c.cell_id for c in motos_free]}")
        print(f"  Celdas ocupadas para MOTO   ({len(motos_occ)}) : {[c.cell_id for c in motos_occ]}")
        print()

    def park_vehicle(self, vehicle, entry_time=None):
        if vehicle.license_plate in self.tickets:
            print(f"⚠️ Alerta: La placa {vehicle.license_plate} ya está registrada.")
            return None
        free = self.available_cells(vehicle.vehicle_type)
        if not free:
            print(f"No hay celdas disponibles para {vehicle.vehicle_type}.")
            return None
        cell = free[0]
        cell.occupied = True
        cell.vehicle = vehicle
        entry = entry_time or datetime.now()
        ticket = Ticket(vehicle, cell, entry)
        self.tickets[vehicle.license_plate] = ticket
        print(f"✅ Registro exitoso para la placa {vehicle.license_plate}.")
        ticket.pretty_print(exit=False)
        self.report_status()
        return ticket

    def release_vehicle(self, license_plate, exit_time=None, payment_method='efectivo'):
        ticket = self.tickets.get(license_plate)
        if not ticket:
            print(f"⚠️ Ticket no encontrado para {license_plate}.")
            return
        ticket.exit_time = exit_time or datetime.now()
        ticket.paid = True
        ticket.payment_method = payment_method
        cell = ticket.cell
        cell.occupied = False
        cell.vehicle = None
        ticket.pretty_print(exit=True)
        self.report_status()
        del self.tickets[license_plate]

# Simulación de ejemplo con múltiples casos

def main():
    operario = Operator(
        id_number="123456789",
        full_name="María Gómez",
        email="mgomez@example.com",
        role="Administrador de Parqueadero"
    )
    print(f"Operario: {operario.full_name} (Cédula: {operario.id_number})")
    print(f"Correo: {operario.email} | Rol: {operario.role}\n")

    lot = ParkingLot(operator=operario)
    # Reporte inicial antes de operaciones
    lot.report_status()

    base_time = datetime.now().replace(second=0, microsecond=0)

    vehicles = [
        Vehicle("ABC-123", 'carro', 'Toyota', 'Rojo'),
        Vehicle("XYZ-789", 'carro', 'BMW', 'Negro'),
        Vehicle("MOTO-01", 'moto', 'Honda', 'Azul'),
        Vehicle("MOTO-02", 'moto', 'Yamaha', 'Blanco'),
        Vehicle("ABC-123", 'carro', 'Toyota', 'Rojo'),
    ]
    intervals = [0, 2, 5, 7, 10]

    for v, mins in zip(vehicles, intervals):
        lot.park_vehicle(v, entry_time=base_time + timedelta(minutes=mins))

    departures = [
        ("MOTO-01", 15, 'efectivo'),
        ("ABC-123", 30, 'tarjeta'),
        ("MOTO-02", 45, 'transferencia'),
        ("XYZ-789", 60, 'efectivo')
    ]

    for plate, mins, method in departures:
        lot.release_vehicle(plate,
                            exit_time=base_time + timedelta(minutes=mins),
                            payment_method=method)

if __name__ == "__main__":
    main()


Operario: María Gómez (Cédula: 123456789)
Correo: mgomez@example.com | Rol: Administrador de Parqueadero

Estado actual del parqueadero:
  Celdas disponibles para CARRO (10) : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  Celdas ocupadas para CARRO  (0) : []
  Celdas disponibles para MOTO  (6) : [11, 12, 13, 14, 15, 16]
  Celdas ocupadas para MOTO   (0) : []

✅ Registro exitoso para la placa ABC-123.

—— TIQUETE DE INGRESO ——
Tipo Vehículo : Carro
Placa         : ABC-123
Marca         : Toyota
Color         : Rojo
Celda         : 1 (carro)
Entrada       : 21:35
———————————————

Estado actual del parqueadero:
  Celdas disponibles para CARRO (9) : [2, 3, 4, 5, 6, 7, 8, 9, 10]
  Celdas ocupadas para CARRO  (1) : [1]
  Celdas disponibles para MOTO  (6) : [11, 12, 13, 14, 15, 16]
  Celdas ocupadas para MOTO   (0) : []

✅ Registro exitoso para la placa XYZ-789.

—— TIQUETE DE INGRESO ——
Tipo Vehículo : Carro
Placa         : XYZ-789
Marca         : BMW
Color         : Negro
Celda         : 2 (carro)
Entr